In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision.datasets as datasets
from torchvision import datasets, models
import torchvision.transforms as T
from tqdm import tqdm
import glob
from pandas.core.common import flatten
import cv2
import gc
import torchvision

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

in_channel = 3
num_classes = 2338
learning_rate = 1e-4
batch_size = 4
num_epochs = 3

In [3]:
model = torchvision.models.efficientnet_v2_s(weights=torchvision.models.EfficientNet_V2_S_Weights.DEFAULT)

# If you want to do finetuning then set requires_grad = False
# Remove these two lines if you want to train entire model,
# and only want to load the pretrain weights.
# letting the entire model to train is best suited for large datasets!!!

for param in model.parameters():
    param.requires_grad = False

model.avgpool = nn.Identity()
model.classifier = nn.Sequential(
    nn.Linear(1280, 576), nn.ReLU(), nn.Linear(576, num_classes)
)
model.to(device)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

In [30]:
# transforms_1k = torchvision.models.EfficientNet_V2_S_Weights.IMAGENET1K_V1.transforms()

transforms_1k = torch.nn.Sequential(
        torchvision.models.EfficientNet_V2_S_Weights.IMAGENET1K_V1.transforms()
)

# scripted_transforms = torch.jit.script(transforms)

In [5]:
####################################################
#       Create Train and Test sets
####################################################
train_data_path = r'S:\Logo2k+ for classification\train_and_test\train\A' 
test_data_path = r'S:\Logo2k+ for classification\train_and_test\test\A'

train_image_paths = [] #to store image paths in list
classes = [] #to store class values

#1.
# get all the paths from train_data_path and append image paths and class to to respective lists
# eg. train path-> 'images/train/26.Pont_du_Gard/4321ee6695c23c7b.jpg'
# eg. class -> 26.Pont_du_Gard
for data_path in glob.glob(train_data_path + '/*'):
    classes.append(data_path.split('\\')[-1]) 
    train_image_paths.append(glob.glob(data_path + '/*'))
    
train_image_paths = list(flatten(train_image_paths))

# random.shuffle(train_image_paths)

print('train_image_path example: ', train_image_paths[0])
print('class example: ', classes[0])

# #2.
# # split train valid from train paths (80,20)
# train_image_paths, valid_image_paths = train_image_paths[:int(0.8*len(train_image_paths))], train_image_paths[int(0.8*len(train_image_paths)):] 

#3.
# create the test_image_paths
test_image_paths = []
for data_path in glob.glob(test_data_path + '/*'):
    test_image_paths.append(glob.glob(data_path + '/*'))

test_image_paths = list(flatten(test_image_paths))

print(f"Train size: {len(train_image_paths)}\nTest size: {len(test_image_paths)}")

train_image_path example:  S:\Logo2k+ for classification\train_and_test\train\A\100 anos\10.jpg
class example:  100 anos
Train size: 116871
Test size: 50164


In [6]:
#######################################################
#      Create dictionary for class indexes
#######################################################

idx_to_class = {i:j for i, j in enumerate(classes)}
class_to_idx = {value:key for key,value in idx_to_class.items()}

In [36]:
#######################################################
#               Define Dataset Class
#######################################################

class LogoDataset(Dataset):
    def __init__(self, image_paths, transform=False):
        self.image_paths = image_paths
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_filepath = self.image_paths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        label = image_filepath.split('\\')[-2]
        label = class_to_idx[label]
        if self.transform is not None:
            image = T.ToTensor()(image)
            image = self.transform(image)
        
        return image, label
    
#######################################################
#                  Create Dataset
#######################################################

train_dataset = LogoDataset(train_image_paths,transforms_1k)
test_dataset = LogoDataset(test_image_paths,transforms_1k)

In [37]:
print('The shape of tensor for 50th image in train dataset: ',train_dataset[49][0].shape)
print('The label for 50th image in train dataset: ',train_dataset[49][1])

The shape of tensor for 50th image in train dataset:  torch.Size([3, 384, 384])
The label for 50th image in train dataset:  1


In [38]:
im = train_dataset[49][0]
im.shape

torch.Size([3, 384, 384])

In [39]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [42]:
# # Train Network
# for epoch in range(num_epochs):
#     losses = []
# #     if epoch%7 == 0:
# #         learning_rate *= 0.1

#     for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
#         # Get data to cuda if possible
#         data = data.to(device=device)
#         targets = targets.to(device=device)

#         # forward
#         scores = model(data)
#         loss = criterion(scores, targets)

#         losses.append(loss.item())
#         # backward
#         optimizer.zero_grad()
#         loss.backward()

#         # gradient descent or adam step
#         optimizer.step()

#     print(f"Cost at epoch {epoch} is {sum(losses)/len(losses):.5f}")

# # Check accuracy on training & test to see how good our model


def check_accuracy(loader, model):
#     if loader.dataset.train:
#         print("Checking accuracy on training data")
#     else:
#         print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            print(x.shape)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


check_accuracy(test_loader, model)

torch.Size([4, 3, 384, 384])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (4x184320 and 1280x576)

In [ ]:
gc.collect()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
print("torch.cuda.memory_allocated: %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024))
print("torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))

In [53]:
import glob
  
files = glob.glob(r'S:\Logo2k+ for classification\train/**/*', 
                   recursive = True)

for file in files:
    if len(file.split('\\')) > 5:
        print(file)


S:\Logo2k+ for classification\train\BACCARAT\Baccarat\1.jpg
S:\Logo2k+ for classification\train\BACCARAT\Baccarat\11.jpg
S:\Logo2k+ for classification\train\BACCARAT\Baccarat\14.jpg
S:\Logo2k+ for classification\train\BACCARAT\Baccarat\15.jpg
S:\Logo2k+ for classification\train\BACCARAT\Baccarat\16.jpg
S:\Logo2k+ for classification\train\BACCARAT\Baccarat\17.jpg
S:\Logo2k+ for classification\train\BACCARAT\Baccarat\18.jpg
S:\Logo2k+ for classification\train\BACCARAT\Baccarat\19.jpg
S:\Logo2k+ for classification\train\BACCARAT\Baccarat\24.jpg
S:\Logo2k+ for classification\train\BACCARAT\Baccarat\26.jpg
S:\Logo2k+ for classification\train\BACCARAT\Baccarat\27.jpg
S:\Logo2k+ for classification\train\BACCARAT\Baccarat\28.jpg
S:\Logo2k+ for classification\train\BACCARAT\Baccarat\29.jpg
S:\Logo2k+ for classification\train\BACCARAT\Baccarat\32.jpg
S:\Logo2k+ for classification\train\BACCARAT\Baccarat\33.jpg
S:\Logo2k+ for classification\train\BACCARAT\Baccarat\35.jpg
S:\Logo2k+ for classifica

In [49]:
len(files)

52575